
## Mathematical Formulation for the Model Agency Optimization Problem

### Decision Variable:
Let $x_{ij} $ be a binary variable where $ x_{ij} = 1 $ if model  $i$ is assigned to job $j$, and $x_{ij} = 0 $ otherwise.

### Auxiliary Variables:
Let $e_{ij}, a_{ij}, n_{ij} $ for each model $i$ is assigned to job $j$ be binary variables for the ethnicity, age category, and non-compete agreement constraints respectively (see Constraints 4,5,6). If either of these auxiliary variables are 1, then the corresponding constraint is binding.

### Parameters:
1. **Base Payment**: $B_{ij} $ representing the **base** payment (i.e. *'Pay Ranger Lower'* in the jobs datset) for assigning model $i$ to job $j$.
2. **Payment Range**: $R_{ij} $ representing the **range** of payment (i.e. *'Pay Ranger Upper' - 'Pay Range Lower'* in the jobs dataset) for assigning model $i$ to job $j$.
3. **Reputation Score**: $R_{i}$ denoting the reputation score for model $i $, based on the previous work of the model.
4. **Experience Score**: $E_i $ for model $i$, based on the number of years worked in the industry.
5. **Following Score**: $F_i $ for model $i$, based on the number of Instagram followers a model has.
6. **CO2 Emissions**: $ C_{ij} $ for assigning model $ i $ to job $ j $.



### Objective Function:
Maximize the total benefit, which is a combination of payments, reputation, experience, following scores, and CO2 emissions. 

The objective function is:
$$
\text{Maximize } 
\sum_{i, j} [B_{ij} + \frac{R_{ij}}{10} \cdot (\alpha R_{i} + \beta E_i + \gamma F_i) - \delta C_{ij}] \cdot x_{ij}
$$


*Where*, 

$ \alpha, \beta,$ and $ \gamma, $ are weight factors for reputation (0.7), experience (0.25), and following scores (0.05) respectively.

$ \delta $ is the penalty factor for CO2 emissions, decided by the management of *EliteToronto*. 


*Note*: The reputation, experience, and following scores are scaled to be between 0 and 10.


### Constraints:

1. **Non-Conflicting Assignments Constraint**:
   
   For each model and each day of the week, the model can be assigned to at most one job that requires availability on that day.
   $$
   \sum_{j \in \text{{Jobs}}_\text{{day}}} x_{ij} \leq 1, \quad \forall i \in \text{{Models}}, \text{{day}} \in \text{{Week}}
   $$

2. **One Model Per Job Constraint**:
   
   Each job can be assigned to at most one model.
   $$
   \sum_{i \in \text{{Models}}} x_{ij} \leq 1, \quad \forall j \in \text{{Jobs}}
   $$

3. **One Job Per Model Constraint**:
   
   Each model can be assigned to at most one job.
   $$
   \sum_{j \in \text{{Jobs}}} x_{ij} \leq 1, \quad \forall i \in \text{{Models}}
   $$

4. **Ethnicity Constraints**:

   Let $e_{ij}$ be a binary parameter that is 1 if the ethnicity of model $i$ matches the ethnicity preference of job $j$ or if the job preference is 'open', and 0 otherwise.

   For all models $i$ and all jobs $j$:

   $$
   x_{ij} \leq e_{ij} \quad \forall i \in \text{Models}, \forall j \in \text{Jobs}
   $$

5. **Age Category Preference Constraints**:

   Let $a_{ij}$ be a binary parameter that is 1 if the age category of model $i$ matches the age category preference of job $j$ or if the job preference is 'open', and 0 otherwise.

   For all models $i$ and all jobs $j$:

   $$
   x_{ij} \leq a_{ij} \quad \forall i \in \text{Models}, \forall j \in \text{Jobs}
   $$

### Extensions:

1. **Non-Compete Agreement**:

   If a model has a non-compete agreement with a client type, the model cannot be assigned to any other job with the same client type.

   For each model $ i $ and each unique client type $ c $ in the set of all client types $ C $, the non-compete constraint is:

   $$
   \sum_{j \in \text{Jobs}|c_j=c} x_{ij} \leq n_{ic} \quad \forall i \in \text{Models}, \forall c \in C
   $$

   where $ c_j $ is the client type of job $ j $, and $ n_{ic} $ is a binary variable which is 1 if model $ i $ does not have a non-compete agreement for client type $ c $, and 0 otherwise.

2. **Job-Package**:
   In reality, some jobs do not pay for the model's travel and accomodation. The revised objective function to incorporate the job-package costs would be:

   $$
   \text{Maximize } 
   \sum_{i, j} \left[B_{ij} + \frac{R_{ij}}{10} \cdot (\alpha R_{i} + \beta E_i + \gamma F_i)  - (1 - k_j) \cdot (D_j R_j + T_{ij}) - \delta C_{ij}\right] \cdot x_{ij}
   $$

   where 
   - $ k_j $ is a binary variable which is 1 if the job $ j $ pays for the model's travel and accomodation, and 0 otherwise. 
   - $ D_j $ is the duration of the shoot for job $j$ (in days).
   - $ R_j $ is the average cost of cost of accomdation per night in the city of job $j$.
   - $ T_{ij} $ is the air travel cost from the model $i$'s home city to job $j$'s location.


### Solution:
The solution to this optimization problem will provide the optimal assignment of models to jobs, maximizing the total benefit while adhering to the above constraints. The result will be a set of binary values for $ x_{ij} $ indicating which model is assigned to which job.


In [23]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB
from tabulate import tabulate


In [24]:
# Load data
def load_data():
    file_path = 'final_data.xlsx' # Path to data file
    model_data = pd.read_excel(file_path, sheet_name='Model Data')
    jobs_data = pd.read_excel(file_path, sheet_name='Jobs Data')
    co2_data = pd.read_excel(file_path, sheet_name='CO2 Data - Airplane')
    return model_data, jobs_data, co2_data


In [25]:
# Calculate reputation score
def calculate_reputation_score(history, scores):
    total_score = sum(scores.get(rank, 0) for rank in history.split(','))
    return total_score / len(history.split(',')) if history else 0

# Calculate following scores
def calculate_following_scores(followers, max_followers):
    return round((followers / max_followers) * 10, 1) if max_followers else 0

# Normalize experience years
def normalize_experience(model_data):
    model_data['Normalized Experience'] = model_data['Years of Experience'].apply(lambda x: (x / 11) * 9 + 1)

# Function to check day availability for feasibility
def is_feasible(model_row, job_row):
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    for day in days_of_week:
        if job_row[day] == 1 and model_row[day] == 0:
            return False
    return True


In [26]:
def initialize_decision_variables(model, model_data, jobs_data):
    assign_vars = {}


    for model_id, model_row in model_data.iterrows():
        for job_id, job_row in jobs_data.iterrows():
            # Assignment variables
            assign_vars[(model_id, job_id)] = model.addVar(vtype=GRB.BINARY, name=f"assign_{model_id}_{job_id}")


    return assign_vars

In [27]:

# Calculate model pay
def calculate_model_pay(model_row, job_row, scores, max_followers):
    lower_bound_pay = job_row['Pay Range Lower']
    pay_range = job_row['Pay Range Upper'] - lower_bound_pay
    reputation_score = calculate_reputation_score(model_row['Previous Work'], scores) * 0.7
    following_score = calculate_following_scores(model_row['Instagram Followers'], max_followers) * 0.05
    experience_score = model_row['Normalized Experience'] * 0.25
    return lower_bound_pay + (pay_range / 10) * (reputation_score + following_score + experience_score)

# Calculate CO2 emission
def calculate_co2_emission(model_row, job_row, co2_data):
    return co2_data.loc[co2_data['Location'] == model_row['Location'], co2_data.columns[co2_data.columns.str.contains(job_row['Location of Job'])][0]].iloc[0]


In [28]:
def set_objective_function(model, assign_vars, model_data, jobs_data, co2_data, scores, emission_weight=0.001):

    max_followers = max(model_data['Instagram Followers'])

    profit_term = sum(assign_vars[m_id, j_id] * calculate_model_pay(model_data.loc[m_id], jobs_data.loc[j_id], scores, max_followers)
                      for m_id, j_id in assign_vars)
    emission_term = sum(assign_vars[m_id, j_id] * calculate_co2_emission(model_data.loc[m_id], jobs_data.loc[j_id], co2_data)
                        for m_id, j_id in assign_vars)

    model.setObjective(profit_term - emission_weight * emission_term, GRB.MAXIMIZE)






In [29]:

# Add constraints
def add_constraints(model, assign_vars, model_data, jobs_data):
    # Non-conflicting assignments
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    for model_id in model_data.index:
        for day in days_of_week:
            model.addConstr(
                sum(assign_vars.get((model_id, job_id), 0) for job_id, job_row in jobs_data.iterrows() if job_row[day] == 1) <= 1,
                f"non_conflicting_{model_id}_{day}")

    # Each job can be assigned to only one model
    for job_id in jobs_data.index:
        model.addConstr(sum(assign_vars.get((model_id, job_id), 0) for model_id in model_data.index) <= 1, f"one_model_per_job_{job_id}")

    # Ensure one model is not assigned to multiple jobs
    for model_id in model_data.index:
        model.addConstr(sum(assign_vars.get((model_id, job_id), 0) for job_id in jobs_data.index) <= 1, f"OneJobPerModel_{model_id}")


    # Ethnicity constraints
    for model_id, model_row in model_data.iterrows():
        for job_id, job_row in jobs_data.iterrows():
            # Binary parameter indicating if the model's ethnicity matches the job's preference or the job's preference is 'open'
            e_ij = int(job_row['Ethnicity Preference'].strip().lower() == 'open' or 
                    model_row['Ethnicity'].strip().lower() == job_row['Ethnicity Preference'].strip().lower())
            model.addConstr(assign_vars[(model_id, job_id)] <= e_ij, f"eth_match_{model_id}_{job_id}")

    # Age Category Preference constraints
    for model_id, model_row in model_data.iterrows():
        for job_id, job_row in jobs_data.iterrows():
            # Binary parameter indicating if the model's age category matches the job's preference or the job's preference is 'open'
            a_ij = int(job_row['Age Category Preference'].strip().lower() == 'open' or 
                    model_row['Age Category'].strip().lower() == job_row['Age Category Preference'].strip().lower())
            model.addConstr(assign_vars[(model_id, job_id)] <= a_ij, f"age_pref_match_{model_id}_{job_id}")

    # Ethnicity constraints
    for job_id, job_row in jobs_data.iterrows():
        job_pref = job_row['Ethnicity Preference'].strip().lower()
        for model_id, model_row in model_data.iterrows():
            model_ethnicity = model_row['Ethnicity'].strip().lower()
            if job_pref != 'open' and model_ethnicity != job_pref:
                model.addConstr(assign_vars.get((model_id, job_id), 0) == 0, f"Ethnicity_match_{model_id}_{job_id}")




In [30]:

# Create optimization model
def create_optimization_model(model_data, jobs_data, co2_data, scores):
    model = Model("ModelAgencyOptimization")
    assign_vars = initialize_decision_variables(model, model_data, jobs_data)
    set_objective_function(model, assign_vars, model_data, jobs_data, co2_data, scores)
    add_constraints(model, assign_vars, model_data, jobs_data)
    return model

In [31]:
def load_travel_and_rate_data():
    package_data = pd.read_excel('final_data.xlsx', sheet_name='Package')
    travel_costs = {}
    for index, row in package_data.iterrows():
        departure_city = row['Departure City']
        travel_costs[departure_city] = {
            'Montreal': row['Travel Cost to Montreal (CAD)'],
            'Toronto': row['Travel Cost to Toronto (CAD)'],
            'Vancouver': row['Travel Cost to Vancouver (CAD)']
        }
    average_rate_per_night = {
        'Montreal': 144,
        'Toronto': 149,
        'Vancouver': 160
    }
    return travel_costs, average_rate_per_night


def calc_travel_cost(model_row, job_row):
    travel_costs, average_rate_per_night = load_travel_and_rate_data()
    if job_row['Job Package'] == 'No':
        duration = job_row['Duration of Shoot']
        travel_cost = travel_costs[model_row['Location']][job_row['Location of Job']]  # Assuming travel_costs is a nested dictionary with travel costs
        rate_per_night = average_rate_per_night[job_row['Location of Job']]
        total_cost = duration * rate_per_night + travel_cost
        return total_cost
    else:
        return 0

In [32]:
# Extract results
def extract_results(model, model_data, jobs_data, co2_data, scores):
    assignments_data = []
    max_followers = max(model_data['Instagram Followers'])
    for var in model.getVars():
        if var.x > 0.5:
            model_id, job_id = map(int, var.varName.split('_')[1:])
            model_row = model_data.loc[model_id]
            job_row = jobs_data.loc[job_id]
            job_pay = calculate_model_pay(model_row, job_row, scores, max_followers)
            travel_cost = calc_travel_cost(model_row, job_row)
            total_profit = job_pay - travel_cost
            model_pay = total_profit * 0.8
            agency_profit = total_profit * 0.2
            co2_emission = calculate_co2_emission(model_row, job_row, co2_data)
            assignments_data.append({
                "Model Name": model_row['Name'],
                "Model Location": model_row['Location'], 
                "Job Name": job_row['Client Name'],
                "Job Location": job_row['Location of Job'],
                "Job Package": job_row['Job Package'],
                "Total Job Pay": round(job_pay,2),
                "Total Travel Cost": round(travel_cost,2),
                "Model Net Income": round(model_pay,2),
                "Agency Profit": round(agency_profit, 2), 
                "CO2 Emissions": co2_emission
            })

    # print unassigned jobs
    for job_id in jobs_data.index:
        if not any(var.x > 0.5 for var in model.getVars() if var.varName.split('_')[2] == str(job_id)):
            job_row = jobs_data.loc[job_id]
            assignments_data.append({
                "Model Name": "Unassigned", 
                "Model Location": "N/A", 
                "Job Name": job_row['Client Name'], 
                "Job Location": job_row['Location of Job'],
                "Job Package": job_row['Job Package'],
                "Total Job Pay": f"{job_row['Pay Range Lower']} - {job_row['Pay Range Upper']}",
                "Total Travel Cost": "N/A",
                "Model Net Income": 0,
                "Agency Profit": 0, 
                "CO2 Emissions": "N/A"
            })
    return pd.DataFrame(assignments_data)

# Print results
def print_results(assignments_df):
    print(tabulate(assignments_df, headers='keys', tablefmt='grid'))
    assignments_df.to_excel('final_results.xlsx', index=False)



In [33]:
# Main execution
def main():
    model_data, jobs_data, co2_data = load_data()
    normalize_experience(model_data)
    scores = {"S": 10, "A": 8, "B": 6}

    model = create_optimization_model(model_data, jobs_data, co2_data, scores)
    model.Params.LogToConsole = 0
    model.optimize()
    assignments_df = extract_results(model, model_data, jobs_data, co2_data, scores)
    print_results(assignments_df)

    

if __name__ == "__main__":
    main()

+----+---------------------+------------------+------------------------+----------------+---------------+-----------------+---------------------+--------------------+-----------------+-----------------+
|    | Model Name          | Model Location   | Job Name               | Job Location   | Job Package   | Total Job Pay   | Total Travel Cost   |   Model Net Income |   Agency Profit | CO2 Emissions   |
+====+=====================+==================+========================+================+===============+=================+=====================+====================+=================+=================+
|  0 | Brodie Scott        | Tokyo            | Frank And Oak          | Vancouver      | Yes           | 5149.35         | 0                   |            4119.48 |         1029.87 | 639             |
+----+---------------------+------------------+------------------------+----------------+---------------+-----------------+---------------------+--------------------+-----------------+----